In [1]:
import utils
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DIR_PATH = "Data"
DIRPATH_EXTRACTED_FEATURES = "extracted_features"

In [2]:
class CustomDataset():
    def __init__(self, brands: list, filenames_features: list, filename_target=utils.FILENAME_RELEVANCE_WINDOW, log_target=False, device=device):
        self.log_target = log_target
        tensor = torch.concat([torch.concat([torch.load(os.path.join(DIRPATH_EXTRACTED_FEATURES, brand, filename_feature), map_location=device, weights_only=False) 
                                for filename_feature in filenames_features], dim=1)
                                  for brand in brands], dim=0)
        numpy_array = tensor.detach().numpy()
        features = pd.DataFrame(numpy_array)

        tensor = torch.concat([torch.load(os.path.join(DIRPATH_EXTRACTED_FEATURES, brand, filename_target), map_location=device, weights_only=False) 
                                  for brand in brands], dim=0)
        numpy_array = tensor.detach().numpy()
        self.target = pd.DataFrame(numpy_array)
        if log_target:
              self.target = np.log(self.target)

        self.features_train, self.features_test, self.target_train, self.target_test = train_test_split(features, self.target, test_size = 0.1)
    
    def __len__(self):
        return len(self.target)

class CustomRegressor():
    def __init__(self, range=None):
        if range: # gave errors when range was zero
            if range[0] == 0:
                range[0] += 1
        self.range = range
        self.regressor = RandomForestRegressor(
            n_estimators=50,
            criterion='squared_error',
            max_depth=None,
            min_samples_leaf=5,
            oob_score=True,
            random_state=0,
            )
        
    def fit(self, Data: CustomDataset):
        target_range = (Data.target_train[0].min(), Data.target_train[0].max() + 1)
        if self.range:
            target_range = self.range
            if Data.log_target:
                target_range = np.log(target_range)
        target_train = Data.target_train.rename(columns={0:'target'})
        df = pd.concat([Data.features_train, target_train], axis=1)
        df = df[df['target'] >= target_range[0]]
        df = df[df['target'] < target_range[1]]
        features_train = df.drop(columns=['target'])
        target_train = df['target']
        self.regressor.fit(features_train, target_train)

    def oob_score(self):
        print(f'Out-of-Bag Score: {self.regressor.oob_score_}')
        
    def score(self, Data: CustomDataset):
        target_range = (Data.target_test[0].min(), Data.target_test[0].max() + 1)
        if self.range:
            if Data.log_target:
                target_range = np.log(self.range)

        target_test = Data.target_test.rename(columns={0:'target'})
        df = pd.concat([Data.features_test, target_test], axis=1)
        df = df[df['target'] >= target_range[0]]
        df = df[df['target'] < target_range[1]]
        features_test = df.drop(columns=['target'])
        target_test = df['target']

        predictions = self.regressor.predict(features_test)
        self.oob_score()
        mse = mean_squared_error(target_test, predictions)
        r2 = r2_score(target_test, predictions)
        print(f'Mean Squared Error: {mse}')
        print(f'R-squared: {r2}')

    def predict_train(self, Data: CustomDataset):
        return self.regressor.predict(Data.features_train)

    def predict_test(self, Data: CustomDataset):
        return self.regressor.predict(Data.features_test)
    
    def predict(self, Data):
        return self.regressor.predict(Data)
        

class CustomClassifier():
    def __init__(self, bins):
        self.bins = bins
        self.classifier = RandomForestClassifier(
            n_estimators=100,
            criterion='gini',
            max_depth=None,
            min_samples_leaf=20,
            oob_score=True,
            random_state=0,
        )
        
    def fit(self, Data: CustomDataset):
        if Data.log_target:
            train_bins = (Data.target_train).map(np.exp)
        else:
            train_bins = Data.target_train
        train_bins = pd.cut(train_bins[0], bins=self.bins, labels=[i for i in range(len(self.bins) - 1)])
        self.classifier.fit(Data.features_train, train_bins)

    def oob_score(self):
        print(f'Out-of-Bag Score: {self.classifier.oob_score_}')
        
    def score(self, Data: CustomDataset):
        if Data.log_target:
            train_bins = (Data.target_train).map(np.exp)
            test_bins = (Data.target_test).map(np.exp)
        else:
            train_bins = Data.target_train
            test_bins = Data.target_test
        train_bins = pd.cut(train_bins[0], bins=self.bins, labels=[i for i in range(len(self.bins) - 1)])
        test_bins = pd.cut(test_bins[0], bins=self.bins, labels=[i for i in range(len(self.bins) - 1)])

        target_train_pred = self.classifier.predict(Data.features_train)
        target_test_pred = self.classifier.predict(Data.features_test)
        train_accuracy = accuracy_score(train_bins, target_train_pred)
        test_accuracy = accuracy_score(test_bins, target_test_pred)

        self.oob_score()
        print("Train Accuracy: %.1f%%" %(train_accuracy * 100))
        print("Test Accuracy: %.1f%%" %(test_accuracy * 100))

    def predict_train(self, Data: CustomDataset):
        return self.classifier.predict(Data.features_train)

    def predict_test(self, Data: CustomDataset):
        print(Data.target_test)
        return self.classifier.predict(Data.features_test)
    
    def predict(self, Data):
        return self.classifier.predict(Data)

In [3]:
filenames_features = [
    utils.FILENAME_BRAND_OHE,
    utils.FILENAME_PUBLICATION_TIMESTAMP,
    utils.FILENAME_PUBLICATION_WEEKDAY_OHE,
    utils.FILENAME_NUM_WORDS,
    utils.FILENAME_NUM_PARAGRAPH,
    # utils.FILENAME_MAIN_SECTION_OHE,
    # utils.FILENAME_SUBSECTIONS_OHE,
    # utils.FILENAME_LDA_TOPICS,
    # utils.FILENAME_IPTC_TOPICS_LEVEL_0,
    # utils.FILENAME_IPTC_TOPICS_LEVEL_1,
]

brands = utils.ALL_BRANDS
# brands = ['ad']

In [ ]:
Data = CustomDataset(brands, filenames_features, log_target=False)
Data.features_train.shape

In [ ]:
target_pred = [24 for _ in range(len(Data.target_test))]
print(mean_squared_error(Data.target_test, target_pred))
print(r2_score(Data.target_test, target_pred))

In [362]:
%%script false --no-raise-error # to skip cell, simple forests 

regressor = CustomRegressor()
regressor.fit(Data)
classifier = CustomClassifier(bins=[0,50,100,1000])
classifier.fit(Data)
print('regressor:')
regressor.score(Data)
print('classifier:')
classifier.score(Data)

y_train = pd.DataFrame(Data.target_train)
y_pred_train = pd.DataFrame(regressor.predict_train(Data))
abs_dif_train = (y_pred_train - y_train).map(np.abs)

y_test = pd.DataFrame(Data.target_test)
y_pred_test = pd.DataFrame(regressor.predict_test(Data))
abs_dif_test = (y_pred_test - y_test).map(np.abs)

In [ ]:
bins = [0, 10, 20, 25, 50, 100, 250, 1000]


regressors = []
for n, lower in enumerate(bins[:-1]):
    regressors.append(CustomRegressor([lower, bins[n + 1]]))

regressors = [] # testing something
for i in range(len(bins) - 2):
    lower = bins[i - 1]
    higher = bins[i + 2]
    if i == 0:
        lower = bins[i]
    if i == len(bins):
        higher = bins[i + 1]
    regressors.append(CustomRegressor([lower, higher]))

classifier = CustomClassifier(bins)

for regressor in regressors:
    regressor.fit(Data)
    print(f'Regressor {regressor.range} trained')

classifier.fit(Data)
print('Classifier trained')

def class_regression(features):
    bin_predictions = classifier.predict(features)

    final_predictions = []
    for i, bin_idx in enumerate(bin_predictions):
        final_predictions.append(regressors[bin_idx].predict(features.iloc[[i]])[0])

    return pd.DataFrame(final_predictions, columns=[0])

final_predictions_test = class_regression(Data.features_test)
print("Final test predictions made")

final_predictions_train = class_regression(Data.features_train)
print("Final train predictions made")

y_train = pd.DataFrame(Data.target_train)
y_pred_train = final_predictions_train
abs_dif_train = (y_pred_train - y_train).map(np.abs)

y_test = pd.DataFrame(Data.target_test)
y_pred_test = final_predictions_test
abs_dif_test = (y_pred_test - y_test).map(np.abs)

In [ ]:
print(mean_squared_error(Data.target_test, final_predictions_test))
print(r2_score(Data.target_test, final_predictions_test))

print('classifier:')
classifier.score(Data)

for regressor in regressors:
    print(f'regressor {regressor.range}:')
    regressor.score(Data)

In [ ]:
print(f'diff mean: {abs_dif_train.mean()}, diff std: {abs_dif_train.std()}')
print(f'diff mean: {abs_dif_test.mean()}, diff std: {abs_dif_test.std()}')
plt.hist(abs_dif_train, bins=50, alpha=0.5, color='b')
plt.title('Difference of the prediction and real training data')
plt.show()
plt.hist(abs_dif_test, bins=50, alpha=0.5, color='b')
plt.title('Difference of the prediction and real test data')
plt.show()

In [ ]:
n_bins = 50
bins = np.histogram(np.hstack((y_pred_train,y_train)), bins=n_bins)[1]
plt.hist(y_pred_train, bins=bins, alpha=0.5, color='b', label='train prediction')
plt.hist(y_train, bins=bins, alpha=0.5, color='r', label='train ground truth')
plt.legend()
plt.show()

bins = np.histogram(np.hstack((y_pred_test,y_test)), bins=n_bins)[1]
plt.hist(y_pred_test, bins=bins, alpha=0.5, color='b', label='test prediction')
plt.hist(y_test, bins=bins, alpha=0.5, color='r', label='test ground truth')
plt.legend()
plt.show()

if Data.log_target:
    y_pred_train_trans = (y_pred_train).map(np.exp)
    y_train_trans = (y_train).map(np.exp)
    y_pred_test_trans = (y_pred_test).map(np.exp)
    y_test_trans = (y_test).map(np.exp)
else:
    y_pred_train_trans = (y_pred_train).map(np.log)
    y_train_trans = (y_train).map(np.log)
    y_pred_test_trans = (y_pred_test).map(np.log)
    y_test_trans = (y_test).map(np.log)

bins = np.histogram(np.hstack((y_pred_train_trans,y_train_trans)), bins=n_bins)[1]
plt.hist(y_pred_train_trans, bins=bins, alpha=0.5, color='b', label='train prediction')
plt.hist(y_train_trans, bins=bins, alpha=0.5, color='r', label='train ground truth')
plt.legend()
plt.show()

bins = np.histogram(np.hstack((y_pred_test_trans,y_test_trans)), bins=n_bins)[1]
plt.hist(y_pred_test_trans, bins=bins, alpha=0.5, color='b')
plt.hist(y_test_trans, bins=bins, alpha=0.5, color='r')
plt.show()